# Taller 2: Aprendizaje Supervisado – Modelos de Clasificación
Integrantes: Zada Riquelme y Vania Reyes.

------------------------------------------------------------------------------------------------------------------------

# Actividad 1
Para el preprocesamiento

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Ver el número de valores faltantes por columna
valores_faltantes = datos.isnull().sum()

# Identificar las columnas categóricas
columnas_categoricas = datos.select_dtypes(include=['object']).columns

# Codificar las variables categóricas
datos_codificados = pd.get_dummies(datos, columns=columnas_categoricas, drop_first=True)

# Normalizar las características numéricas
columnas_numericas = datos_codificados.select_dtypes(include=['float64', 'int64']).columns
escalador = StandardScaler()
datos_codificados[columnas_numericas] = escalador.fit_transform(datos_codificados[columnas_numericas])

# Separar características y la variable objetivo
X = datos_codificados.drop(columns=['Rotulo_Titulo_SI'])
y = datos_codificados['Rotulo_Titulo_SI']

# Dividir los datos en entrenamiento y prueba (solo para referencia)
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Crear el clasificador RandomForest
clf = RandomForestClassifier(random_state=42)

# Definir el número de folds para k-Fold CV
k = 5

# Realizar validación cruzada k-Fold estratificada
validacion_cruzada = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Para almacenar los índices de entrenamiento y validación de cada pliegue
indices_pliegues = []

# Generar los índices para cada pliegue
for indice_entrenamiento, indice_validacion in validacion_cruzada.split(X_entrenamiento, y_entrenamiento):
    indices_pliegues.append((indice_entrenamiento, indice_validacion))

# Imprimir los índices de entrenamiento y validación del primer pliegue como ejemplo
indice_entrenamiento, indice_validacion = indices_pliegues[0]
print("Índices de entrenamiento:", indice_entrenamiento)
print("Índices de validación:", indice_validacion)


Índices de entrenamiento: [   1    2    3 ... 2411 2412 2413]
Índices de validación: [   0    4    8   11   16   18   19   29   31   37   44   45   65   68
   69   74   75   81   85   89   90   98  100  101  105  106  108  123
  127  128  131  137  149  150  158  162  171  181  184  200  201  205
  207  208  213  214  224  225  228  234  236  243  252  255  256  258
  261  268  280  282  289  290  293  296  299  303  310  317  319  321
  327  331  342  346  348  349  350  358  364  370  371  387  394  404
  406  412  413  417  419  421  432  442  459  463  464  477  481  483
  485  487  503  512  515  523  526  532  534  541  563  565  567  571
  572  576  578  579  581  588  589  607  611  612  614  621  626  627
  629  632  637  641  646  648  651  652  669  677  680  684  686  688
  694  698  711  716  727  731  736  739  742  743  750  752  753  759
  768  770  791  794  801  803  806  819  826  829  835  838  841  847
  854  857  859  861  876  877  878  882  885  886  894  895  8

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Definir el clasificador RandomForest
rf_clf = RandomForestClassifier(random_state=42)

# Definir los parámetros a ajustar
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20]
}

# Definir la estrategia de validación cruzada k-Fold estratificada
k = 5
stratified_kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Realizar búsqueda exhaustiva de parámetros con validación cruzada estratificada
grid_search_rf = GridSearchCV(rf_clf, param_grid_rf, cv=stratified_kfold, scoring='accuracy')
grid_search_rf.fit(X_entrenamiento, y_entrenamiento)

# Obtener el mejor modelo y su puntaje
best_rf_model = grid_search_rf.best_estimator_
best_rf_score = grid_search_rf.best_score_

print("Mejor modelo RandomForestClassifier:")
print(best_rf_model)
print(f"Puntuación de precisión en validación cruzada: {best_rf_score}")



Mejor modelo RandomForestClassifier:
RandomForestClassifier(max_depth=20, n_estimators=150, random_state=42)
Puntuación de precisión en validación cruzada: 0.7580904272226661


In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Definir el clasificador
svm_clf = SVC(random_state=42)

# Definir los parámetros a ajustar
param_grid_svm = {
    'C': [1, 10, 100],
    'kernel': ['linear', 'rbf']
}

# Realizar búsqueda exhaustiva de parámetros
grid_search_svm = GridSearchCV(svm_clf, param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm.fit(X_entrenamiento, y_entrenamiento)

# Obtener el mejor modelo y su puntaje
best_svm_model = grid_search_svm.best_estimator_
best_svm_score = grid_search_svm.best_score_

print("Mejor modelo SVM:")
print(best_svm_model)
print(f"Puntuación de precisión en validación cruzada: {best_svm_score}")


In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Cargar el conjunto de datos de evaluación
archivo_evaluacion = 'Taller_2_Titulacion_Evaluación.csv'
datos_evaluacion = pd.read_csv(archivo_evaluacion, encoding='latin-1', delimiter=';')

# Suponiendo que `preprocessor` es el objeto que utilizaste para preprocesar `X_train`
# y `best_rf_model` es el mejor modelo RandomForestClassifier seleccionado

# Preprocesar datos de evaluación similar al conjunto de entrenamiento
X_evaluacion_processed = preprocessor.transform(datos_evaluacion)

# Realizar predicciones sobre el conjunto de evaluación usando el mejor modelo encontrado
y_pred_eval = best_rf_model.predict(X_evaluacion_processed)

# Convertir predicciones a formato 'SÍ' o 'NO'
predicciones_evaluacion_etiquetas = ['SÍ' if pred == 1 else 'NO' for pred in y_pred_eval]

# Crear DataFrame para el archivo de salida
resultado_df = pd.DataFrame({'Id': datos_evaluacion['Id'], 'Predicción': predicciones_evaluacion_etiquetas})

# Guardar resultados en archivo de texto
archivo_salida = 'clasificación_título.txt'
resultado_df.to_csv(archivo_salida, sep=';', index=False, encoding='utf-8')

print(f'Archivo "{archivo_salida}" generado exitosamente.')


NameError: name 'preprocessor' is not defined

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definir el modelo RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

# Definir parámetros para la búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Inicializar GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')

# Realizar la búsqueda de hiperparámetros en el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo seleccionado
best_rf_model = grid_search.best_estimator_

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Mostrar la precisión del mejor modelo en el conjunto de validación cruzada
print("Precisión del mejor modelo en CV:", grid_search.best_score_)


Mejores parámetros encontrados:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Precisión del mejor modelo en CV: 0.761385015850107


In [38]:
import pandas as pd
from sklearn.preprocessing import StandardScaler  # Importa el preprocesador adecuado según tu caso

# Cargar los datos de evaluación
archivo_evaluacion = 'Taller_2_Titulacion_Evaluación.csv'
datos_evaluacion = pd.read_csv(archivo_evaluacion, encoding='latin-1', delimiter=';')

# Asumiendo que `preprocessor` es el objeto que utilizaste para preprocesar `X_train`
# y `best_rf_model` es el mejor modelo RandomForestClassifier seleccionado

# Preprocesar datos de evaluación similar al conjunto de entrenamiento
# Ajusta según tus datos y preprocesador utilizado
scaler = StandardScaler()  # Ejemplo de preprocesador, ajusta según tu caso
X_evaluacion_numerico = datos_evaluacion.select_dtypes(include=['int64', 'float64'])  # Selecciona características numéricas
X_evaluacion_numerico_scaled = scaler.fit_transform(X_evaluacion_numerico)  # Ajusta y transforma según tu preprocesador

# Asume que aquí también se manejan las variables categóricas si es necesario

# Concatenar características preprocesadas
X_evaluacion_processed = X_evaluacion_numerico_scaled  # Ajusta según tus datos y preprocesador

# Realizar predicciones sobre el conjunto de evaluación usando el mejor modelo encontrado
y_pred_eval = best_rf_model.predict(X_evaluacion_processed)

# Convertir predicciones a formato 'SÍ' o 'NO'
predicciones_evaluacion_etiquetas = ['SÍ' if pred == 1 else 'NO' for pred in y_pred_eval]

# Crear DataFrame para el archivo de salida
resultado_df = pd.DataFrame({'Id': datos_evaluacion['Id'], 'Predicción': predicciones_evaluacion_etiquetas})

# Guardar resultados en archivo de texto
archivo_salida = 'clasificación_título.txt'
resultado_df.to_csv(archivo_salida, sep=';', index=False, encoding='utf-8')

print(f'Archivo "{archivo_salida}" generado exitosamente.')


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 46 features, but RandomForestClassifier is expecting 493 features as input.